<a href="https://colab.research.google.com/github/MacUpr/im_ia/blob/main/Imers%C3%A3o_Agentes_de_IA_Alura_%2B_Google_Gemini_Aula_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries for document loading, splitting, and vector storage
!pip install -q --upgrade \
    langchain_community \
    faiss-cpu \
    langchain-text-splitters \
    pymupdf

In [ ]:
from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader

docs = []

# Iterate through all PDF files in the /content/ directory
for n in Path("/content/").glob("*.pdf"):
    try:
        # Load each PDF document using PyMuPDFLoader
        loader = PyMuPDFLoader(str(n))
        docs.extend(loader.load())
        print(f"Carregado com sucesso arquivo {n.name}")
    except Exception as e:
        # Print an error message if a file fails to load
        print(f"Erro ao carregar arquivo {n.name}: {e}")

# Print the total number of documents loaded
print(f"Total de documentos carregados: {len(docs)}")

Carregado com sucesso arquivo Política de Reembolsos (Viagens e Despesas).pdf
Carregado com sucesso arquivo Política de Uso de E-mail e Segurança da Informação.pdf
Carregado com sucesso arquivo Políticas de Home Office.pdf
Total de documentos carregados: 3


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Initialize the RecursiveCharacterTextSplitter with a chunk size of 300 and overlap of 30
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)

# Split the loaded documents into smaller chunks
chunks = splitter.split_documents(docs)

In [ ]:
# Iterate through each chunk and print its content and metadata
for chunk in chunks:
    print(chunk)
    print("------------------------------------")

page_content='Política de Reembolsos (Viagens e 
Despesas) 
 
1.​ Reembolso: requer nota fiscal e deve ser submetido em até 10 dias corridos após a 
despesa.​
 
2.​ Alimentação em viagem: limite de R$ 70/dia por pessoa. Bebidas alcoólicas não 
são reembolsáveis.​' metadata={'producer': 'Skia/PDF m140 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': '/content/Política de Reembolsos (Viagens e Despesas).pdf', 'file_path': '/content/Política de Reembolsos (Viagens e Despesas).pdf', 'total_pages': 1, 'format': 'PDF 1.4', 'title': 'Imersão: Política de Reembolsos (Viagens e Despesas)', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}
------------------------------------
page_content='são reembolsáveis.​
 
3.​ Transporte: táxi/app são permitidos quando não houver alternativa viável. 
Comprovantes obrigatórios.​
 
4.​ Internet para home office: reembolsável via subsídio mensal de até R$ 100, 
conforme p

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# Create Google Generative AI Embeddings
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-001",
    google_api_key=GOOGLE_API_KEY # Assuming GOOGLE_API_KEY is defined elsewhere
)

In [ ]:
from langchain_community.vectorstores import FAISS

# Create a FAISS vector store from the document chunks and embeddings
vectorstore = FAISS.from_documents(chunks, embeddings)

# Create a retriever from the vector store with a similarity score threshold
retriever = vectorstore.as_retriever(search_type="similarity_score_threshold",
                                     search_kwargs={"score_threshold":0.3, "k": 4})

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

# Create a chat prompt template with system and human messages
prompt_rag = ChatPromptTemplate.from_messages([
    ("system",
     "Você é um Assistente de Políticas Internas (RH/IT) da empresa Carraro Desenvolvimento. "
     "Responda SOMENTE com base no contexto fornecido. "
     "Se não houver base suficiente, responda apenas 'Não sei'."),

    ("human", "Pergunta: {input}\n\nContexto:\n{context}")
])

# Create a document chain to combine the retrieved documents with the prompt
document_chain = create_stuff_documents_chain(llm_triagem, prompt_rag)

In [ ]:
# Helper functions for formatting the output
import re, pathlib
from typing import List, Dict

def _clean_text(s: str) -> str:
    # Remove extra whitespace from the text
    return re.sub(r"\s+", " ", s or "").strip()

def extrair_trecho(texto: str, query: str, janela: int = 240) -> str:
    # Extract a relevant snippet from the text based on the query
    txt = _clean_text(texto)
    termos = [t.lower() for t in re.findall(r"\w+", query or "") if len(t) >= 4]
    pos = -1
    for t in termos:
        pos = txt.lower().find(t)
        if pos != -1: break
    if pos == -1: pos = 0
    ini, fim = max(0, pos - janela//2), min(len(txt), pos + janela//2)
    return txt[ini:fim]

def formatar_citacoes(docs_rel: List, query: str) -> List[Dict]:
    # Format citations from the retrieved documents
    cites, seen = [], set()
    for d in docs_rel:
        src = pathlib.Path(d.metadata.get("source","")).name
        page = int(d.metadata.get("page", 0)) + 1
        key = (src, page)
        if key in seen:
            continue
        seen.add(key)
        cites.append({"documento": src, "pagina": page, "trecho": extrair_trecho(d.page_content, query)})
    return cites[:3]

In [ ]:
def perguntar_politica_RAG(pergunta: str) -> Dict:
    # Retrieve relevant documents based on the user's question
    docs_relacionados = retriever.invoke(pergunta)

    # If no relevant documents are found, return "Não sei"
    if not docs_relacionados:
        return {"answer": "Não sei.",
                "citacoes": [],
                "contexto_encontrado": False}

    # Use the document chain to generate an answer based on the retrieved documents
    answer = document_chain.invoke({"input": pergunta,
                                    "context": docs_relacionados})

    txt = (answer or "").strip()

    # If the generated answer is "Não sei", return "Não sei"
    if txt.rstrip(".!?") == "Não sei":
        return {"answer": "Não sei.",
                "citacoes": [],
                "contexto_encontrado": False}

    # Otherwise, return the answer and formatted citations
    return {"answer": txt,
            "citacoes": formatar_citacoes(docs_relacionados, pergunta),
            "contexto_encontrado": True}

In [ ]:
# Define a list of test questions to query the RAG system
testes = ["Posso reembolsar a internet?",
          "Quero mais 5 dias de trabalho remoto. Como faço?",
          "Posso reembolsar cursos ou treinamentos da Alura?",
          "Quantas capivaras tem no Rio Pinheiros?",
          "Qual a política de reembolso para alimentação em viagem?",
          "Preciso usar VPN quando trabalho de casa?"]

In [ ]:
# Iterate through each test question and get the answer from the RAG system
for msg_teste in testes:
    resposta = perguntar_politica_RAG(msg_teste)
    print(f"PERGUNTA: {msg_teste}")
    print(f"RESPOSTA: {resposta['answer']}")
    if resposta['contexto_encontrado']:
        print("CITAÇÕES:")
        # Print citations if relevant context was found
        for c in resposta['citacoes']:
            print(f" - Documento: {c['documento']}, Página: {c['pagina']}")
            print(f"   Trecho: {c['trecho']}")
        print("------------------------------------")

PERGUNTA: Posso reembolsar a internet?
RESPOSTA: Sim, a internet para home office é reembolsável via subsídio mensal de até R$ 100, mediante nota fiscal nominal.
CITAÇÕES:
 - Documento: Política de Reembolsos (Viagens e Despesas).pdf, Página: 1
   Trecho: lsáveis.​ 3.​ Transporte: táxi/app são permitidos quando não houver alternativa viável. Comprovantes obrigatórios.​ 4.​ Internet para home office: reembolsável via subsídio mensal de até R$ 100, conforme política de Home Office.​
 - Documento: Políticas de Home Office.pdf, Página: 1
   Trecho: 5.​ Conectividade: há subsídio mensal de internet domiciliar para quem trabalha em home office: até R$ 100/mês, mediante nota fiscal nominal.​ 6.​ Solicitação de
------------------------------------
PERGUNTA: Quero mais 5 dias de trabalho remoto. Como faço?
RESPOSTA: Para solicitar mais 5 dias de trabalho remoto, você deve formalizar a solicitação via chamado ao RH com a justificativa do seu gestor.
CITAÇÕES:
 - Documento: Políticas de Home Offi

This cell installs the necessary libraries for this notebook: `langchain_community`, `faiss-cpu`, `langchain-text-splitters`, and `pymupdf`. These libraries are used for loading and processing PDF documents and creating a vector store for RAG.